In [3]:
import sys
sys.path.append('../utils/')
from parsing import *
import pandas as pd
import re

import warnings
warnings.filterwarnings("ignore")

PATH_INTERMEDIATE = '../../data/intermediate/'
PATH_RAW = '../../data/raw/'
PATH_FINAL = '../../data/final/'

## OZP

In [41]:
insurance_companies = pd.DataFrame()

In [42]:
ozp = pd.read_excel(PATH_RAW+'ic/ozp.xlsx')

In [43]:
ozp.columns = ['year', 'facility_id', 'expertise', 'count'] # rok, identifikacni cislo strediska, odbornost, pocet vykonu
ozp['insurance_company'] = 'OZP'
ozp.info()
insurance_companies = insurance_companies.append(ozp)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343327 entries, 0 to 343326
Data columns (total 5 columns):
year                 343327 non-null int64
facility_id          343327 non-null int64
expertise            343325 non-null object
count                343327 non-null int64
insurance_company    343327 non-null object
dtypes: int64(3), object(2)
memory usage: 13.1+ MB


## VZP

In [44]:
vzp = pd.read_excel(PATH_RAW+'ic/vzp.xlsx')

In [45]:
vzp.columns = ['year', 'expertise', 'procedure_code', 'region', 'count'] # rok, odbornost, kod vykonu, region, pocet vykonu
vzp['insurance_company'] = 'VZP'
vzp.info()
insurance_companies = insurance_companies.append(vzp)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455390 entries, 0 to 455389
Data columns (total 6 columns):
year                 455390 non-null int64
expertise            455390 non-null object
procedure_code       455390 non-null int64
region               455390 non-null object
count                455390 non-null int64
insurance_company    455390 non-null object
dtypes: int64(3), object(3)
memory usage: 20.8+ MB


## SKODA

In [46]:
skoda = pd.read_csv(PATH_RAW+'ic/skoda.csv', sep=';', usecols=['odb','sum_vykony','ico','rok'])

In [47]:
skoda.columns = ['expertise', 'count', 'facility_id', 'year'] # rok, odbornost, kod vykonu, region, pocet vykonu
skoda['insurance_company'] = 'SKODA'
skoda.info()
insurance_companies = insurance_companies.append(skoda)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80502 entries, 0 to 80501
Data columns (total 5 columns):
expertise            80491 non-null object
count                80502 non-null int64
facility_id          80435 non-null float64
year                 80502 non-null int64
insurance_company    80502 non-null object
dtypes: float64(1), int64(2), object(2)
memory usage: 3.1+ MB


In [36]:
insurance_companies.rename(columns={'expertise':'expertise_code'})

insurance_companies.reset_index().to_feather(PATH_INTERMEDIATE+'insurance_companies.ftr')
insurance_companies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 879219 entries, 0 to 80501
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   count              879219 non-null  int64  
 1   expertise_code     879206 non-null  object 
 2   facilities_id      423762 non-null  float64
 3   insurance_company  879219 non-null  object 
 4   procedure_code     455390 non-null  float64
 5   region             455390 non-null  object 
 6   year               879219 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 53.7+ MB


## Codes name

In [2]:
ic = pd.read_feather(PATH_INTERMEDIATE+'insurance_companies.ftr').set_index('index')

ic = ic.rename(columns={'expertise':'expertise_code',
                        'facilities_id' : 'ico'})


In [5]:
with open(PATH_RAW+'ic/ODBORN', 'r', encoding="utf-8") as f:
    codes = pd.DataFrame([x.split(';') for x in f.read().splitlines()], columns=['expertise_code', 'expertise_name'])

In [6]:
# set(codes.expertise_code_code.unique()).difference(set(ic.expertise_code.unique()))
unknown_exp = set(ic.expertise_code.unique()).difference(set(codes.expertise_code.unique()))
unknown_count = ic[ic.expertise_code.isin(unknown_exp)]['count'].sum()
total_count = ic['count'].sum()

print(f'Počet nepřiřazených záznamů: {ic[ic.expertise_code.isin(unknown_exp)].shape[0]}/{ic.shape[0]} ({round(100*ic[ic.expertise_code.isin(unknown_exp)].shape[0]/ic.shape[0],2)} %)')
print(f'Počet výkonů z nepřiřazených záznamů: {unknown_count}/{total_count} ({round(100*unknown_count/total_count,2)} %)')

Počet nepřiřazených záznamů: 53154/879219 (6.05 %)
Počet výkonů z nepřiřazených záznamů: 347225075/4953258809 (7.01 %)


In [19]:
ic = ic[~ic.expertise_code.isin(unknown_exp) & (ic['count'] > 75)]
ic_total = pd.merge(ic,codes, on=['expertise_code'])

In [52]:
ic_total.to_csv(PATH_FINAL+'insurances.csv', index=True)

,count,expertise_code,ico,insurance_company,procedure_code,region,year,expertise_name
0,122,001,69909041.0,OZP,NaN,None,2012,praktické lékařství pro dospělé
1,170,001,67011080.0,OZP,NaN,None,2015,praktické lékařství pro dospělé
2,198,001,24235164.0,OZP,NaN,None,2016,praktické lékařství pro dospělé
3,247,001,18231454.0,OZP,NaN,None,2016,praktické lékařství pro dospělé
4,952,001,60700904.0,OZP,NaN,None,2011,praktické lékařství pro dospělé
...,...,...,...,...,...,...,...,...
442375,592,7U5,195201.0,SKODA,NaN,None,2021,dlouhodobá (chronická) ústavní lůžková péče o...
442376,710,7U5,195201.0,SKODA,NaN,None,2017,dlouhodobá (chronická) ústavní lůžková péče o...
442377,1197,7U5,195201.0,SKODA,NaN,None,2019,dlouhodobá (chronická) ústavní lůžková péče o...
442378,984,7U5,195201.0,SKODA,NaN,None,2020,dlouhodobá (chronická) ústavní lůžková péče o...


### Time and expenses

In [38]:
# https://www.vzp.cz/poskytovatele/ciselniky/zdravotni-vykony
# https://szv.mzcr.cz/Vykon?page=0

dial_df = pd.DataFrame()
for i in range(81):
    x = pd.read_html(f'https://szv.mzcr.cz/Vykon?page={i}') # 10.11.2022
    dial_df = dial_df.append(x[0])

dial_df = dial_df.drop_duplicates()
dial_df = dial_df[['Odbornost', 'Číslo', 'Název', 'Doba trvání', 'Omezení frekvencí', 'Celkem']]
dial_df.columns = ['expertise_code', 'procedure_code', 'procedure_name', 'time_in_mins', 'frequency', 'total_cost']
dial_df.expertise_code = dial_df.expertise_code.apply(lambda n:f'{n:03}')

In [64]:
import tabula
df = tabula.read_pdf('https://media.vzpstatic.cz/media/Default/dokumenty/ciselniky/vykony_01348.pdf', pages='all')

In [74]:
dial_df = pd.concat(df)
dial_df = dial_df.drop_duplicates()
new_header = dial_df.iloc[0]
dial_df = dial_df[1:]
dial_df.columns = new_header
dial_df

,KOD,ODB,OME,NAZ,VYS,ZUM,TVY,CTN,PMA,PJP,BOD,KAT,NaN,NaN,NaN
1,00041,000,A,JEDEN DEN POBYTU PACIENTA VE STACIONÁŘI I.,"Vykazuje se, pokud délka pobytu dosáhne alespo...",NaN,480.0,49.0,0,NaN,175,NaN,NaN,NaN,NaN
2,00042,000,A,JEDEN DEN POBYTU PACIENTA VE STACIONÁŘI II.,"Vykazuje se, za 1 den pobytu pacienta ve staci...",NaN,180.0,25.0,0,NaN,87,NaN,NaN,NaN,NaN
3,00130,304,S,DLOUHODOBÁ MIMOTĚLNÍ RESPIRAČNÍ PODPORA\rNOVOR...,Dlouhodobá mimotělní respirační podpora ECLS (...,Z,480.0,1920.0,0,NaN,18033,NaN,NaN,NaN,NaN
4,00132,304,SA,VYŠETŘENÍ DÍTĚTE S VÝZNAMNÝM PERINATÁLNÍM RIZIKEM,Specializované komplexní klinické vyšetření dí...,NaN,70.0,70.0,0,NaN,959,NaN,NaN,NaN,NaN
5,00301,000,D,"OD TYPU 01 - PRO NEMOCNICE TYPU 0, (KATEGORIE 3)",Vykazuje se na akutních lůžkách interních obor...,NaN,0.0,237.0,0,NaN,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44,NaN,NaN,NaN,NaN,(VZP) KÓD POUZE PRO PRACOVIŠTĚ SMLUVNÍ ODB. 128 -,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,NaN,NaN,NaN,NaN,SLOUŽÍ PRO VYKÁZÁNÍ ERYTROPOETINU VYDANÉHO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,Lze vykázat pouze s kontrolním nebo cíleným vy...,99992,999,NaN,NaN,L,0.0,0.0,NaN,0,NaN,0,NaN,NaN,NaN
47,NaN,NaN,NaN,NaN,PRACOVIŠTĚM PACIENTOVI S PERITONEÁLNÍ DIALÝZOU K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## NRPZS

In [16]:
# nrpzs = pd.read_csv('https://opendata.mzcr.cz/data/nrpzs/narodni-registr-poskytovatelu-zdravotnich-sluzeb.csv') # 25. 9. 2022
nrpzs = pd.read_csv(PATH_RAW+'nrpzs.csv')

nrpzs.columns = [re.sub(r"(?<=[a-z])([A-Z])", r"_\1", x).lower() for x in nrpzs.columns]

not_required = ['Lékárna',
                'Samostatné zařízení fyzioterapeuta',
                'Zdravotnická zachranná služba',
                'Výdejna zdravotnických prostředků']
nrpzs = nrpzs[~nrpzs.druh_zarizeni.isin(not_required)].reset_index(drop=True)

nrpzs = nrpzs[[x for x in nrpzs.columns if not 'poskytovatel_' in x]]


nrpzs.loc[nrpzs.okres == 'území Hlavního města Prahy', 'okres'] = 'Hlavní město Praha'
nrpzs.loc[nrpzs.okres.isna() & (nrpzs.kraj == 'Hlavní město Praha'), 'okres'] = 'Hlavní město Praha'
nrpzs.loc[nrpzs.kraj.isna() & (nrpzs.okres == 'Hlavní město Praha'), 'kraj'] = 'Hlavní město Praha'
nrpzs.loc[nrpzs.kraj.isna() & (nrpzs.okres == 'Náchod'), 'kraj'] = 'Královéhradecký kraj'
nrpzs.loc[nrpzs.kraj.isna() & (nrpzs.okres == 'Pelhřimov'), 'kraj'] = 'Kraj Vysočina'

nrpzs.to_csv(PATH_FINAL+'nrpzs.csv')